In [1]:
from google.colab import drive
import pandas as pd
import numpy as np


In [7]:
import threading
import time


In [3]:
drive.mount('/content/drive')
%cd /content/drive/MyDrive/LCB proj

!ls

Mounted at /content/drive
/content/drive/MyDrive/LCB proj
 clashsimulator.py		    LCB_skill_spec.csv
'Clash Win Rate Simulator.ipynb'    LCB_skill_spec.gsheet
 datas_clash.csv		    machine_learning.ipynb
 example_sample			    __pycache__
 example_sample.csv		    temp_data.csv
'GoogleColab Excel Convert.ipynb'  'threading example.ipynb'


In [4]:
import clashsimulator as Clash

In [5]:
skill_informations = pd.read_csv("/content/drive/MyDrive/LCB proj/LCB_skill_spec.csv")

In [6]:
skill_informations

,캐릭터,인격 등급,인격,스킬 이름,스킬 위력,스킬 코인 갯수,코인 위력,비고
0,이상,1.0,LCB 수감자,쳐내기,3,1,7,NaN
1,NaN,NaN,NaN,밀어 찌르기,4,2,4,NaN
2,NaN,NaN,NaN,연격,6,3,1,NaN
3,NaN,2.0,세븐 협회 6과,플레쉬,4,1,6,NaN
4,NaN,NaN,NaN,리포스트,5,2,2,NaN
...,...,...,...,...,...,...,...,...
121,NaN,NaN,NaN,몰아치기,5,3,1,NaN
122,NaN,NaN,NaN,연무 장타,4,4,2,NaN
123,NaN,3.0,G사 일등대리,베어내기,4,2,3,NaN
124,NaN,NaN,NaN,사지절단,6,1,10,NaN


In [8]:
def load_data (data_frame=None):
    character = None
    personality = None

    for i in range(len(skill_informations)):

        if skill_informations.isnull().loc[i]['인격'] == False:
            if personality != None:
                character.append(personality)
            personality = [skill_informations.loc[i]['인격']]
        
        
        if skill_informations.isnull().loc[i]['캐릭터'] == False:
            if character != None:
                #character.append(personality)
                skill_data.append(character)
            character = [skill_informations.loc[i]['캐릭터']]
            
        personality.append(
            [
                skill_informations.loc[i]['스킬 이름'],
                skill_informations.loc[i]['스킬 위력'],
                skill_informations.loc[i]['스킬 코인 갯수'],
                skill_informations.loc[i]['코인 위력']
            ]
        )
    character.append(personality)
    skill_data.append(character)

In [9]:
global learn_data 
learn_data = pd.DataFrame(columns = ['skill force', 'coin count', 'coin force', 'coin probability', 
                                     'skill force_foreign', 'coin count_foreign', 'coin force_foreign', 'coin probability_foreign',
                                     'win rate'])

In [26]:
global learn_data_temp 
learn_data_temp = pd.DataFrame(columns = ['skill force', 'coin count', 'coin force', 'coin probability', 
                                     'skill force_foreign', 'coin count_foreign', 'coin force_foreign', 'coin probability_foreign',
                                     'win rate'])

In [11]:
def data_gen():
    for i in range(1):
        for q in range(1, 2): #임시 data extration용 
    #for i in range(len(skill_data)-1):
    #    for q in range(i+1, len(skill_data)):    
            global learn_data_temp

            character1 = skill_data[i][1:]
            character2 = skill_data[q][1:]
            character1_name = skill_data[i][0]
            character2_name = skill_data[q][0]

            for personality1 in character1:
                for personality2 in character2:
                    person1 = personality1[1:]
                    person2 = personality2[1:]

                    for skillA in person1:
                        for skillB in person2:
                            print(personality1[0], skillA, personality2[0], skillB)
                            skillforceA = []
                            coincountA = []
                            coinforceA = []
                            coinprobabilityA = []
                            skillforceB = []
                            coincountB = []
                            coinforceB = []
                            coinprobabilityB = []
                            winrate = []

                            Clash_result = Clash.get_skill_prob_list(skillA[1:], skillB[1:], clash_repeat, mentality_repeat)
                            for cnt in range(extract_quantity):
                                x_ = int(np.random.uniform(mentality_repeat))
                                y_ = int(np.random.uniform(mentality_repeat))
                                x = Clash_result['x'][x_]
                                y = Clash_result['y'][y_]
                                z = Clash_result['z'][x_][y_]

                                skillforceA.append(skillA[1])
                                coincountA.append(skillA[2])
                                coinforceA.append(skillA[3])
                                coinprobabilityA.append(x)
                                skillforceB.append(skillB[1])
                                coincountB.append(skillB[2])
                                coinforceB.append(skillB[3])
                                coinprobabilityB.append(y)
                                winrate.append(z)


                            data_temp = {
                                'skill force'                 : skillforceA,
                                'coin count'                  : coincountA,
                                'coin force'                  : coinforceA,
                                'coin probability'            : coinprobabilityA,
                                'skill force_foreign'         : skillforceB,
                                'coin count_foreign'          : coincountB,
                                'coin force_foreign'          : coinforceB,
                                'coin probability_foreign'    : coinprobabilityB,
                                'win rate'                    : winrate
                            }

                            learn_data_temp = pd.concat([learn_data_temp, pd.DataFrame(data_temp)])
                            
            print(character1_name + " : " + character2_name +"  ::  data extract completed.")

In [19]:
def multiple_generation(mode="general"):
    global  whole_number 
    process_list = list()
    if mode == "temp":
        whole_number = 6
        for i in range(3):
            for q in range(i+1, 4):
                character1_name = skill_data[i][0]
                character2_name = skill_data[q][0]

                p = threading.Thread(target=generation_part, args=(character1_name, character2_name, mode))
                p.start()
                process_list.append(p)

    else:
        whole_number = 66
        for i in range(11):
            for q in range(i+1, 12):
                character1_name = skill_data[i][0]
                character2_name = skill_data[q][0]

                p = threading.Thread(target= generation_part, args=(character1_name, character2_name, mode))
                p.start()
                process_list.append(p)


    for th in process_list:
        th.join()


In [20]:
def generation_part(characterA, characterB, mode="General"):
    global learn_data_temp
    global learn_data
    global processcnt
    global whole_number

    character1_index = character_to_id[characterA]
    character2_index = character_to_id[characterB]

    character1_name = skill_data[character1_index][0]
    character2_name = skill_data[character2_index][0] 

    character1_inform = skill_data[character1_index][1:]
    character2_inform = skill_data[character2_index][1:] 

    for personality_data1 in character1_inform:
        for personality_data2 in character2_inform:

            personality1_name = personality_data1[0]
            personality2_name = personality_data2[0]

            personality1_skills = personality_data1[1: ]
            personality2_skills = personality_data2[1: ]
            print(character1_name+ "-"+ personality1_name + "  ::  "+ character2_name + "-" + personality2_name)

            for skillA in personality1_skills:
                for skillB in personality2_skills:
                    #print(character1_name+"_"+personality1_name, skillA, character2_name+"_"+personality2_name, skillB)
                    skillA_name = skillA[0]
                    skillB_name = skillB[0]
                    
                    skillA_data = skillA[1:]
                    skillB_data = skillB[1:]

                    skillforceA ,coincountA ,coinforceA  ,coinprobabilityA ,skillforceB  ,coincountB   ,coinforceB  ,coinprobabilityB ,winrate = [list()for i in range(9)]

                    Clash_result = Clash.get_skill_prob_list(skillA_data, skillB_data, clash_repeat, mentality_repeat)
                    

                    for cnt in range(extract_quantity):
                        x_ = int(np.random.uniform(mentality_repeat))
                        y_ = int(np.random.uniform(mentality_repeat))
                        x = Clash_result['x'][x_]
                        y = Clash_result['y'][y_]
                        z = Clash_result['z'][x_][y_]

                        
                        skillforceA.append(skillA_data[0])
                        coincountA.append(skillA_data[1])
                        coinforceA.append(skillA_data[2])
                        coinprobabilityA.append(x)
                        skillforceB.append(skillB_data[0])
                        coincountB.append(skillB_data[1])
                        coinforceB.append(skillB_data[2])
                        coinprobabilityB.append(y)
                        winrate.append(z*100) #z값은 본래 0에서 1 사이이지만, 100을 곱해 퍼센트로 표시함 (오차를 늘리기 위함)

                    
                    data_temp = {
                        'skill force'                 : skillforceA,
                        'coin count'                  : coincountA,
                        'coin force'                  : coinforceA,
                        'coin probability'            : coinprobabilityA,
                        'skill force_foreign'         : skillforceB,
                        'coin count_foreign'          : coincountB,
                        'coin force_foreign'          : coinforceB,
                        'coin probability_foreign'    : coinprobabilityB,
                        'win rate'                    : winrate
                    }
                    aquired = lock.acquire()
                    if mode == "temp":
                        learn_data_temp = pd.concat([learn_data_temp, pd.DataFrame(data_temp)])
                        
                    else:
                        learn_data = pd.concat([learn_data, pd.DataFrame(data_temp)])
                    lock.release()

                    
    
    processcnt +=1
    print(character1_name + " : " + character2_name +"  ::  data extract completed. ( " + str(processcnt) + " / "+ str(whole_number)+" )")
   


In [14]:
character_to_id = {
    "이상"          : 0,
    "파우스트"      : 1,
    "돈키호테"      : 2, 
    "로슈"          : 3, 
    "뫼르소"        : 4, 
    "홍루"          : 5,
    "히스클리프"    : 6,
    "이스마엘"      : 7,
    "로쟈"          : 8, 
    "싱클레어"      : 9,
    "오티스"        : 10,
    "그레고르"      : 11
}

In [15]:
extract_quantity = 300
clash_repeat = 2
mentality_repeat = 2
#temp값과 real값을 넘나들 때 사용용
file_name = "datas_clash"
skill_data=None
global whole_number
global processcnt

In [28]:
lock = threading.Lock()

if __name__ == '__main__':
    processcnt = 0
    if skill_data == None:
      skill_data = []
      load_data(skill_data)

    multiple_generation(mode="temp")
    learn_data_temp.to_csv(file_name + ".csv")

    
    print("data extract completed.")

이상-LCB 수감자  ::  파우스트-LCB 수감자
이상-LCB 수감자  ::  돈키호테-LCB 수감자
이상-LCB 수감자  ::  로슈-LCB 수감자
파우스트-LCB 수감자  ::  돈키호테-LCB 수감자
파우스트-LCB 수감자  ::  로슈-LCB 수감자
돈키호테-LCB 수감자  ::  로슈-LCB 수감자
이상-LCB 수감자  ::  파우스트-W사 정리 요원이상-LCB 수감자  ::  돈키호테-시 협회 5과
파우스트-LCB 수감자  ::  로슈-세븐 협회 6과

돈키호테-LCB 수감자  ::  로슈-세븐 협회 6과
파우스트-LCB 수감자  ::  돈키호테-시 협회 5과이상-LCB 수감자  ::  로슈-세븐 협회 6과

파우스트-LCB 수감자  ::  로슈-흑운회 와카슈
돈키호테-LCB 수감자  ::  로슈-흑운회 와카슈
이상-LCB 수감자  ::  돈키호테-W사 정리 요원
이상-LCB 수감자  ::  로슈-흑운회 와카슈
이상-LCB 수감자  ::  파우스트-살아남은 로보토미 직원
파우스트-LCB 수감자  ::  돈키호테-W사 정리 요원
파우스트-W사 정리 요원  ::  로슈-LCB 수감자
돈키호테-시 협회 5과  ::  로슈-LCB 수감자
이상-세븐 협회 6과  ::  돈키호테-LCB 수감자
이상-세븐 협회 6과  ::  로슈-LCB 수감자
이상-LCB 수감자  ::  파우스트-쥐는 자
파우스트-W사 정리 요원  ::  돈키호테-LCB 수감자
돈키호테-시 협회 5과  ::  로슈-세븐 협회 6과
파우스트-W사 정리 요원  ::  로슈-세븐 협회 6과
이상-세븐 협회 6과  ::  로슈-세븐 협회 6과
이상-세븐 협회 6과  ::  파우스트-LCB 수감자
이상-세븐 협회 6과  ::  돈키호테-시 협회 5과
파우스트-W사 정리 요원  ::  돈키호테-시 협회 5과
돈키호테-시 협회 5과  ::  로슈-흑운회 와카슈
파우스트-W사 정리 요원  ::  로슈-흑운회 와카슈
이상-세븐 협회 6과  ::  로슈-흑운회 와카슈
이상-세븐 협회 6과  ::  파우스트-W

In [ ]:
skill_data

In [29]:
learn_data_temp
#temp를 입력해 만든 예제 데이터셋. 주어진 상황과 동일한 수의 row가 만들어졌다.

,skill force,coin count,coin force,coin probability,skill force_foreign,coin count_foreign,coin force_foreign,coin probability_foreign,win rate
0,3,1,7,0.666667,3,1,7,0.666667,100.0
1,3,1,7,0.666667,3,1,7,0.666667,100.0
2,3,1,7,0.666667,3,1,7,0.666667,100.0
3,3,1,7,0.666667,3,1,7,0.666667,100.0
4,3,1,7,0.666667,3,1,7,0.666667,100.0
...,...,...,...,...,...,...,...,...,...
295,6,3,2,0.666667,1,5,2,0.666667,100.0
296,6,3,2,0.666667,1,5,2,0.666667,100.0
297,6,3,2,0.666667,1,5,2,0.666667,100.0
298,6,3,2,0.666667,1,5,2,0.666667,100.0
